# Solara

> Solara é uma biblioteca python que permitem a criação de aplicações de dados. Ele é utilizado para exibir as taxas de aprovações do professores.

In [ ]:
#| default_exp solara

In [ ]:
#| exporti
from typing import Union, Tuple

import solara
import pandas as pd
import matplotlib.pyplot as plt

from ufrn_cheat_sheet.storage import ReadOnlyInMemorySQLite

In [ ]:
#| export
#| skip_exec: true
#| skip_showdoc: true
db = ReadOnlyInMemorySQLite()

In [ ]:
#| exporti
def db_get(data: Union[Tuple, solara.Reactive]):
    if data.value and isinstance(data.value, list):
        return data.value[0]

    return data.value

In [ ]:
#| export
unidade = solara.reactive(None)
componente = solara.reactive(None)
docente = solara.reactive(None)

## Componentes

Solara trabalha com o conceito de componentes. A atualização de uma variável reativa atualiza os componentes que a utilizam, dessa forma a mudança de uma unidade modifica a lista de componentes disponíveis, e a escolha de um componente modifica a lista de docentes, por fim, a seleção de um docente apresenta o plot da taxa de aprovação.

In [ ]:
#| export
@solara.component
def SelectUnidade():
    values = db.execute_query('SELECT DISTINCT(unidade_responsavel) from data ORDER BY unidade_responsavel')
    solara.Select(label="Unidade", value=unidade, values=values)

In [ ]:
#| export
@solara.component
def SelectComponente():
    values = db.execute_query(
        f"""
        SELECT DISTINCT(nome_componente) from data 
        WHERE unidade_responsavel = '{db_get(unidade)}' 
        ORDER BY nome_componente
        """
    )
    solara.Select(label="Componente", value=componente, values=values)

In [ ]:
#| export
@solara.component
def SelectDocente():
    values = db.execute_query(
        f"""
        SELECT DISTINCT(nome_docente) from data 
        WHERE 
            unidade_responsavel = '{db_get(unidade)}' 
            AND 
            nome_componente = '{db_get(componente)}' 
        ORDER BY nome_componente
        """
    )
    solara.Select(label="Docente", value=docente, values=values)

In [ ]:
#| export
def PieChart():
    #solara.use_state(docente)
    
    q = f"""SELECT descricao, COUNT(*) as contagem
    FROM (
        SELECT DISTINCT discente, descricao
        FROM data
        WHERE unidade_responsavel = '{db_get(unidade)}'
        AND nome_componente = '{db_get(componente)}'
        AND nome_docente = '{db_get(docente)}'
    )
    GROUP BY descricao;
    """

    data = pd.read_sql_query(q, db.connection)

    data['porcentagem'] = (data['contagem'] / data['contagem'].sum()) * 100

    contagem_descricao = data['descricao'].value_counts()
    porcentagens = (contagem_descricao / contagem_descricao.sum()) * 100
    
    fig, ax = plt.subplots(figsize=(10, 6))
    wedges, texts, autotexts = ax.pie(porcentagens, labels=porcentagens.index, autopct='%1.1f%%', startangle=90, wedgeprops=dict(width=0.4))
    
    for text, autotext in zip(texts, autotexts):
        text.set(size=10)
        autotext.set(size=10)
    
    legend_labels = [f"{label}: {value}" for label, value in zip(contagem_descricao.index, contagem_descricao.values)]
    ax.legend(wedges, legend_labels, title="Descrições", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    solara.FigureMatplotlib(fig)

## Page

Solara trabalha o conceito de `Page()` para combinar os componentes em uma página, estiliza-los e adicionar a estrutura em um layout. Esse é o resultado final da aplicação.

In [ ]:
#| export
@solara.component
def Page():
    with solara.AppBarTitle():
        solara.Text("UFRN Cheat Sheet")

    with solara.Column():
        with solara.Row():
            SelectUnidade()
            SelectComponente()
            SelectDocente()
        PieChart()

In [ ]:
#| export
Page()